# build-cat-ccam_noresm2-mm_historical_aus-10i_12km

### use `ecgtools` to build a custom `intake-esm` catalogue for ccam_noresm2-mm_historical_aus-10i_12km

Date: 19 May 2023

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

#### Reference documents: https://ecgtools.readthedocs.io/en/latest

In [1]:
import glob
import pathlib
import traceback
from datetime import datetime

import xarray as xr

from ecgtools import Builder
from ecgtools.builder import INVALID_ASSET, TRACEBACK

from matplotlib import pyplot as plt

In [2]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 2
Total threads: 2,Total memory: 9.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45937,Workers: 2
Dashboard: /proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 9.00 GiB
Comm: tcp://127.0.0.1:37961,Total threads: 1
Dashboard: /proxy/44583/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:41887,


### get catalogue path from config file

In [3]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the config file
#########
#### you will need to specifiy your correct path the the `data-catalogue/config.ini` file 
#########
config.read('./code/ACS/data-catalogue/config.ini')

# Get the value of a variable
catalogue_path = config.get('paths', 'catalogue_path')

### build the catalogue from the listing of files on `xv83`

In [4]:
root_source_path = '/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/'

In [5]:
files = sorted(glob.glob(root_source_path+'*/*/*'))
len(files)

32964

### Write `ecgtools` parser, <br> see: https://ecgtools.readthedocs.io/en/latest/how-to/use-a-custom-parser.html
#### we don't really need to open all the files, given variable name is in the filename.

In [6]:
def parse_CCAM(file):
    """CCAM data stored in"""
    file = pathlib.Path(file)
    info = {}

    try:
        stem = file.stem
        split = stem.split('_')
        variable = split[0]
        domain = split[1]
        host_GCM = split[2]
        run_type = split[3]
        host_ensemble = split[4]
        downscale_model = split[5]
        downscale_version = split[6]
        period = split[7]
        if period == '1hr':
            time_period = 'hourly'
        elif period == '6hr':
            time_period = 'six_hourly'
        elif period == 'day':
            time_period = 'daily'
        elif period == 'mon':
            time_period = 'monthly'
        else:
            time_period = 'fixed'    

        
        info = {
            'variable': variable,
            'domain': domain,
            'host_GCM':host_GCM,
            'run_type':run_type,
            'host_ensemble':host_ensemble,
            'downscale_model':downscale_model,
            'downscale_version':downscale_version,
            'period': period,
            'time_period': time_period,
            'path': str(file),
        }
        return info
    
    except Exception:
        return {INVALID_ASSET: file, TRACEBACK: traceback.format_exc()}

#### test parser with single file

In [7]:
parse_CCAM(files[0])

{'variable': 'CAPE',
 'domain': 'AUS-10i',
 'host_GCM': 'NCC-NorESM2-MM',
 'run_type': 'historical',
 'host_ensemble': 'r1i1p1f1',
 'downscale_model': 'CSIRO-CCAM-2203',
 'downscale_version': 'v1',
 'period': '1hr',
 'time_period': 'hourly',
 'path': '/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/1hr/CAPE/CAPE_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1_1hr_19510101-19511231.nc'}

# setup builder object

In [8]:
%%time
b = Builder([root_source_path],depth=2)

CPU times: user 1.17 ms, sys: 2.89 ms, total: 4.06 ms
Wall time: 3.95 ms


In [9]:
b

Builder(paths=['/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/'], storage_options={}, depth=2, exclude_patterns=[], include_patterns=[], joblib_parallel_kwargs={})

In [10]:
%%time
b.build(parsing_func = parse_CCAM)

CPU times: user 3.98 s, sys: 2.18 s, total: 6.17 s
Wall time: 23.8 s


/g/data/v14/tm4888/miniconda3/envs/ACS-demo/lib/python3.10/site-packages/ecgtools/builder.py:204: UserWarning: Unable to parse 4 assets. A list of these assets can be found in `.invalid_assets` attribute.
  ).clean_dataframe()


Builder(paths=['/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/'], storage_options={}, depth=2, exclude_patterns=[], include_patterns=[], joblib_parallel_kwargs={})

In [11]:
b.df

,variable,domain,host_GCM,run_type,host_ensemble,downscale_model,downscale_version,period,time_period,path
0,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
1,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
2,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
3,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
4,CAPE,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,1hr,hourly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
...,...,...,...,...,...,...,...,...,...,...
32959,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
32960,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
32961,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...
32962,zmla,AUS-10i,NCC-NorESM2-MM,historical,r1i1p1f1,CSIRO-CCAM-2203,v1,mon,monthly,/g/data/xv83/mxt599/ccam_noresm2-mm_historical...


#### 4 `fx` "fixed frequency" `netcdf` files are excluded as "invalid assets"

In [12]:
b.invalid_assets['INVALID_ASSET'].values

array([PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/orog/orog_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1.nc'),
       PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/sftlaf/sftlaf_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1.nc'),
       PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/sftlf/sftlf_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-CCAM-2203_v1.nc'),
       PosixPath('/g/data/xv83/mxt599/ccam_noresm2-mm_historical_aus-10i_12km/drs_cordex/CORDEX/output/AUS-10i/CSIRO/NCC-NorESM2-MM/historical/r1i1p1f1/CSIRO-CCAM-2203/v1/fx/sfturf/sfturf_AUS-10i_NCC-NorESM2-MM_historical_r1i1p1f1_CSIRO-C

## save your `intake-esm` catalogue

In [13]:
b.save(
    # File path - could save as .csv (uncompressed csv) or .csv.gz (compressed csv)
    name = "ccam_noresm2-mm_historical_aus-10i_12km",
    directory = catalogue_path,
    # Column name including filepath
    path_column_name='path',
    # Column name including variables
    variable_column_name='variable',
    # Data file format - could be netcdf or zarr (in this case, netcdf)
    data_format="netcdf",
    # Which attributes to groupby when reading in variables using intake-esm
    groupby_attrs=["domain", "host_GCM", "run_type","host_ensemble","downscale_model","downscale_version","period"],
    # Aggregations which are fed into xarray when reading in data using intake
    aggregations=[
        {
            "type": "join_existing",
            "attribute_name": "date",
            "options": {"dim": "time", "coords": "minimal", "compat": "override"},
        }
    ],
)

Successfully wrote ESM catalog json file to: file:///g/data/v14/tm4888/code/ACS/data-catalogue/catalogues/ccam_noresm2-mm_historical_aus-10i_12km.json


/jobfs/84612829.gadi-pbs/ipykernel_1209062/1166365249.py:1: UserWarning: Unable to parse 4 assets/files. A list of these assets can be found in /jobfs/84612829.gadi-pbs/ccam_noresm2-mm_historical_aus-10i_12km_invalid_assets.csv.
  b.save(


# THE END

In [14]:
client.shutdown()